In [ ]:
from benlp.llms import Chat
from benlp.tools.code_executor import CodeExecutor, execute_code
import io
import contextlib
from benlp.utils import parse_code_blocks

In [20]:
import arxiv
import ast
import concurrent
from csv import writer
from IPython.display import display, Markdown, Latex
import json
import openai
import os
import pandas as pd
from PyPDF2 import PdfReader
import requests
from scipy import spatial
from tenacity import retry, wait_random_exponential, stop_after_attempt
import tiktoken
from tqdm import tqdm
from termcolor import colored
import time
import random
from scipy.stats import norm

from benlp.tools.code_executor import execute_code

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"


In [ ]:
data_dir = "../../data/test/"

In [2]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [3]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [12]:
# Initiate our get_articles and read_article_and_summarize functions
custom_functions = [
    {
        "name" : "python",
        "description" : """Use this function to execute python code.""",
        "parameters" : {
            "type" : "object",
            "properties" : {
                "code" : {
                    "type" : "string",
                    "description" : f"""
                            Python code to be executed
                            """,
                }
            },
            "required" : ["code"],
        }
    }
]


In [18]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    full_message = response.json()["choices"][0]
    if full_message["finish_reason"] == "function_call":
        print(f"Function generation requested, calling function")
        return call_custom_functions(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response.json()
    
def parse_json(json_string):
    """This function parses a json string and returns the corresponding dictionary"""
    return json.loads(json_string)

def call_custom_functions(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""

    if full_message["message"]["function_call"]["name"] == "python":
        args = full_message["message"]["function_call"]["arguments"].strip()
        print("Arguments: ", args)
        parsed_output = parse_json(args)
        print("Executing python code")
        result = execute_code(parsed_output["code"])
        print("Code executed")
        print(result)
        return result

    else:
        raise Exception(f'Function {full_message["message"]["function_call"]["name"]} does not exist and cannot be called')

## Test

In [25]:
# Start with a system message
system_message = """You are PythonGPT, a helpful assistant that is connected with a python interpreter.
You provide accurate and helpful answers to questions about python code. You always write python code that is syntactically correct. Your JSON is always output in a valid schema. You can use the function "python" to execute python code.
"""
conversation = Conversation()
conversation.add_message("system", system_message)

In [26]:
# Add another user message to induce our system to use the second tool
conversation.add_message(
    "user",
    "You can use the function 'python' to execute python code. Make sure to print whatever you want to persist with a print() function. Calculate the result of black scholes for stock price 100, strike price 95, time to maturity 0.5, risk free rate 0.05, volatility 0.2",  
)
updated_response = chat_completion_with_function_execution(
    conversation.conversation_history, functions=custom_functions
)
print("Updated response: ", updated_response)
display(Markdown(updated_response["choices"][0]["message"]["content"]))

Function generation requested, calling function
Arguments:  from math import log, sqrt, exp
from scipy.stats import norm

S = 100  # stock price
K = 95  # strike price
T = 0.5  # time to maturity
r = 0.05  # risk-free rate
sigma = 0.2  # volatility

# Calculate d1 and d2
d1 = (log(S / K) + (r + sigma**2 / 2) * T) / (sigma * sqrt(T))
d2 = d1 - sigma * sqrt(T)

# Calculate the option price using Black-Scholes formula
call_price = S * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)

call_price


JSONDecodeError: Expecting value: line 1 column 1 (char 0)